In [1]:
!pip install pandas 
!pip install mysql-connector-python
!pip install pymysql
!pip install sqlalchemy
!pip install python-dotenv



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
import pymysql


In [3]:
data_path = '../data/MunE19_c.csv'

In [4]:
# Leer el archivo CSV sin índice
df = pd.read_csv(data_path, sep=';', encoding='ISO-8859-1')

# Crear una nueva columna de índice que comience en 0
df.insert(0, 'index', range(0, len(df)))

# Obtener el nombre del archivo original sin la extensión
file_name, file_extension = os.path.splitext(data_path)

# Crear el nombre del archivo de salida añadiendo "_mod"
data_mod_path = f"{file_name}_mod{file_extension}"

# Guardar el archivo CSV con el nuevo índice
df.to_csv(data_mod_path, index=False)

In [5]:
# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
host = os.getenv('HOST')
port = int(os.getenv('PORT'))
database = os.getenv('DATABASE')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

In [6]:
# Conectarse a la base de datos MySQL
connection = pymysql.connect(host=host, user=user, password=password, port=port, local_infile=True)

try:
    with connection.cursor() as cursor:
        # Crear la base de datos si no existe
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
        cursor.execute(f"USE {database};")

        # Crear la tabla con la estructura adecuada
        create_table_query = """
        CREATE TABLE IF NOT EXISTS mi_tabla (
            `index` INT NOT NULL,
            nombre_columna1 VARCHAR(255),
            nombre_columna2 VARCHAR(255),
            -- otras columnas aquí
            PRIMARY KEY (`index`)
        );
        """
        cursor.execute(create_table_query)

        # Habilitar la opción LOCAL para LOAD DATA INFILE
        connection._allow_local_infile = True

        # Cargar los datos del archivo CSV en la tabla
        load_data_query = f"""
        LOAD DATA LOCAL INFILE '{data_mod_path}'
        INTO TABLE mi_tabla
        FIELDS TERMINATED BY ',' 
        ENCLOSED BY '"' 
        LINES TERMINATED BY '\\n'
        IGNORE 1 ROWS;
        """
        cursor.execute(load_data_query)

    # Confirmar cambios
    connection.commit()
finally:
    # Cerrar la conexión
    connection.close()


OperationalError: (3948, 'Loading local data is disabled; this must be enabled on both the client and server sides')